In [1]:
from neo4j import GraphDatabase
import json
import csv
import ast
import re
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup

URL_DB = "bolt://localhost:7687"  # hoặc IP nếu là máy khác

driver = GraphDatabase.driver(URL_DB, auth=("neo4j", "password"))

In [ ]:
# 4624: Successful logon
# 4625: Logon failure
# 4768: Kerberos Authentication (TGT Request)
# 4769: Kerberos Service Ticket (ST Request)
# 4776: NTLM Authentication
# 4672: Assign special privileges

: 

In [5]:
def login_to_flask(server_url, username, password):
    login_url = f"{server_url}/login"

    try:
        # 1. Lấy CSRF Token
        client = requests.session()  # Sử dụng session để giữ cookie
        response = client.get(login_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        csrf_token = soup.find('input', {'id': 'csrf_token'})['value']

        # 2. Gửi yêu cầu POST để đăng nhập
        login_data = {
            'username': username,
            'password': password,
            'csrf_token': csrf_token,
            'remember': 'y'  # Thêm 'remember' nếu có
        }
        response = client.post(login_url, data=login_data)

        # 3. Kiểm tra đăng nhập thành công
        if response.status_code == 200 and "Invalid username or password" not in response.text:
            print("Đăng nhập thành công!")
            return client  # Trả về đối tượng session
        else:
            print("Đăng nhập thất bại. Kiểm tra username/password/CSRF token.")
            return None

    except Exception as e:
        print(f"Lỗi: {e}")
        return None


def upload_security_evtx(session, server_url, evtx_file_path, timezone, casename, addlog=True, sigmascan=False):
    try:
        if not os.path.exists(evtx_file_path):
            print(f"Lỗi: File EVTX không tồn tại tại {evtx_file_path}")
            return "FAIL", {}

        files = {'file0': open(evtx_file_path, 'rb')}
        data = {
            'timezone': timezone,
            'logtype': 'EVTX',
            'addlog': 'true' if addlog else 'false',
            'sigmascan': 'true' if sigmascan else 'false',
            'casename': casename,
        }

        response = session.post(f"{server_url}/upload", files=files, data=data)
        cookies = response.cookies.get_dict()

        print(response.text)

        if response.text == "SUCCESS":
            return "SUCCESS", cookies
        else:
            print(f"Lỗi từ server: {response.text}")
            return "FAIL", {}

    except Exception as e:
        print(f"Lỗi: {e}")
        return "FAIL", {}

if __name__ == '__main__':
    server_url = "http://localhost:8080"
    username = "neo4j"
    password = "password"

    session = login_to_flask(server_url, username, password)
    if session:
        response = session.get(f"{server_url}/")

        uploads_folder = r"D:\\_test\\logon_tracer\\extend\\uploads"
        timezone = 7
        casename = "neo4j"

        for filename in os.listdir(uploads_folder):
            if filename.lower().endswith(".evtx"):
                evtx_file = os.path.join(uploads_folder, filename)
                result, cookies = upload_security_evtx(session, server_url, evtx_file, timezone, casename)
                print(f"Tải lên {filename}: {result}, vui lòng chờ backend xử lý 😾")
    else:
        print("Không thể đăng nhập.")


Đăng nhập thành công!
SUCCESS
Tải lên bach_Security_bach.evtx: SUCCESS, vui lòng chờ backend xử lý 😾
SUCCESS
Tải lên CCber_Security_CCber.evtx: SUCCESS, vui lòng chờ backend xử lý 😾
SUCCESS
Tải lên duong_Security_duong.evtx: SUCCESS, vui lòng chờ backend xử lý 😾
SUCCESS
Tải lên khiem_Security_khiem.evtx: SUCCESS, vui lòng chờ backend xử lý 😾
SUCCESS
Tải lên PhongDailakimtien_Security_PhongDailakimtien.evtx: SUCCESS, vui lòng chờ backend xử lý 😾
SUCCESS
Tải lên Security.evtx: SUCCESS, vui lòng chờ backend xử lý 😾
SUCCESS
Tải lên vuong_Security_vuong.evtx: SUCCESS, vui lòng chờ backend xử lý 😾


In [3]:
def get_all_nodes(tx):
    query = "MATCH (n) RETURN n"
    result = tx.run(query)
    nodes_data = []
    for record in result:
        node = record["n"]
        properties = dict(node)
        nodes_data.append({"id": node.id, "labels": list(node.labels), "properties": properties})
    return nodes_data

with driver.session() as session:
    all_nodes = session.execute_read(get_all_nodes)

json_output = json.dumps(all_nodes, indent=2)

csv_file_path = "./data/nodes.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["id", "labels", "properties"]  # Xác định các cột
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for node_data in all_nodes:
        writer.writerow(node_data)

driver.close()

print(f"\nDữ liệu nodes đã được lưu vào file: {csv_file_path}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_15480\662155363.py:8: DeprecationWarning: `id` is deprecated, use `element_id` instead
  nodes_data.append({"id": node.id, "labels": list(node.labels), "properties": properties})



Dữ liệu nodes đã được lưu vào file: ./data/nodes.csv


In [4]:
def get_all_relationships(tx):
    query = "MATCH ()-[r]->() RETURN r"
    result = tx.run(query)
    relationships_data = []
    for record in result:
        relationship = record["r"]
        relationships_data.append({
            "id": relationship.id,
            "type": relationship.type,
            "start_node_id": relationship.start_node.id,
            "end_node_id": relationship.end_node.id,
            "properties": dict(relationship)
        })
    return relationships_data

with driver.session() as session:
    all_relationships = session.execute_read(get_all_relationships)

json_output = json.dumps(all_relationships, indent=2)

csv_file_path = "./data/relationships.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["id", "type", "start_node_id", "end_node_id", "properties"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for relationship_data in all_relationships:
        writer.writerow(relationship_data)

driver.close()

print(f"\nData relationships đã được lưu vào file: {csv_file_path}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_15480\3509183309.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\Admin\AppData\Local\Temp\ipykernel_15480\3509183309.py:8: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "id": relationship.id,
C:\Users\Admin\AppData\Local\Temp\ipykernel_15480\3509183309.py:10: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "start_node_id": relationship.start_node.id,
C:\Users\Admin\AppData\Local\Temp\ipykernel_15480\3509183309.py:11: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "end_node_id": relationship.end_node.id,



Data relationships đã được lưu vào file: ./data/relationships.csv
